# Data Initialization

### Create synthetic e-commerce data

In [5]:
import litellm
import pandas as pd
import numpy as np
import dotenv
from io import StringIO
import csv

dotenv.load_dotenv()

True

## Schema

E-commerce database with the following tables:

### Table: products
- product_id: int, PRIMARY KEY, unique product identifier
- product_brand: str, brand name
- product_name: str, product name
- product_description: str, product description
- product_price: float, product price
- product_category: str, main category
- product_subcategory: str, subcategory

### Table: customers
- customer_id: int, PRIMARY KEY, unique customer identifier
- customer_city: str, customer's city
- customer_state: str, customer's state/province
- customer_zip: str, customer's zip code
- customer_country: str, customer's country
- customer_gender: str, customer's gender
- customer_age: int, customer's age
- customer_class_quartile: str, customer's class quartile
- customer_description: str, customer's description

### Table: orders
- order_id: int, part of COMPOSITE PRIMARY KEY, unique order identifier
- order_seq_nbr: int, part of COMPOSITE PRIMARY KEY, sequence number for the order
- customer_id: int, FOREIGN KEY to customers.customer_id
- product_id: int, FOREIGN KEY to products.product_id
- order_date: date, date of the order
- COMPOSITE PRIMARY KEY: (order_id, order_seq_nbr)

## Products dataset generation

In [80]:
# Experiment with a few data points before scaling up
rows = 30
prompt = f'''
Create a synthetic e-commerce products dataset in csv format with {rows} rows.
There's only 3 categories: "Clothing", "Drinks", "Electronics".
There's only 3 subcategories:
"Clothing": "T-shirts", "Pants", "Shoes"
"Drinks": "Soda", "Beer", "Water"
"Electronics": "Laptop", "Smartphone", "Headphones"

Only 3 brands exist.
"Luxor" is the luxury brand. Should be expensive.
"Hermanos" is the middle brand. Should be in the middle of the price range.
"Daily Value" is the budget brand. Should be cheap.

The dataset should include the following columns:
product_id, product_brand, product_name, product_description, product_price, product_category, product_subcategory

Each product should be unique, and should be an actual specific product.
Apple Macbook Pro is not a valid product because it's a generic product.
Only include the csv data in the response with no backticks or other formatting.
'''.strip()
response = litellm.completion(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": prompt
    }]
)

print(response['choices'][0]['message']['content'])


product_id,product_brand,product_name,product_description,product_price,product_category,product_subcategory
1,Luxor,Luxor Premium T-Shirt,High-quality cotton T-shirt with elegant design,49.99,Clothing,T-shirts
2,Luxor,Luxor Luxe Pants,Stylish tailored pants crafted from fine materials,129.99,Clothing,Pants
3,Luxor,Luxor Designer Shoes,Handcrafted leather shoes for formal occasions,199.99,Clothing,Shoes
4,Hermanos,Hermanos Classic T-Shirt,Comfortable and stylish T-shirt for everyday wear,24.99,Clothing,T-shirts
5,Hermanos,Hermanos Casual Pants,Durable pants suited for casual outings,49.99,Clothing,Pants
6,Hermanos,Hermanos Trendy Sneakers,Versatile sneakers great for multiple activities,69.99,Clothing,Shoes
7,Daily Value,Daily Value Basic T-Shirt,Affordable basic T-shirt for all occasions,9.99,Clothing,T-shirts
8,Daily Value,Daily Value Simple Pants,Functional pants at a budget price,19.99,Clothing,Pants
9,Daily Value,Daily Value Low-Cost Sneakers,Practical sneakers that don’t break th

In [81]:
def parse_llm_csv_response(response):
    """Parse CSV response from LLM and return DataFrame with comma handling"""
    try:
        content = response['choices'][0]['message']['content']
        
        # Handle different CSV formats the LLM might return
        if '```csv' in content:
            # Extract CSV from markdown code block
            start = content.find('```csv') + 6
            end = content.find('```', start)
            csv_data = content[start:end].strip()
        elif '```' in content:
            # Extract CSV from regular code block
            start = content.find('```') + 3
            end = content.find('```', start)
            csv_data = content[start:end].strip()
        else:
            # Assume it's raw CSV
            csv_data = content.strip()
        
        # Step 1: Read lines manually
        lines = csv_data.strip().split('\n')
        header = lines[0]
        rows = lines[1:]
        
        # Step 2: Fix comma issues in data (especially in text fields)
        fixed_lines = [header]
        for line in rows:
            cols = line.split(',')
            if len(cols) > 7:  # extra commas breaking the row
                # Try to merge all fields between product_name and product_price
                fixed_row = cols[:3]  # first 3 columns
                # merge product_description (might have commas)
                description_fields = cols[3:len(cols)-3]
                description = ','.join(description_fields).strip()
                if not (description.startswith('"') and description.endswith('"')):
                    description = f'"{description}"'
                fixed_row.append(description)
                fixed_row.extend(cols[-3:])  # last 3 columns
            else:
                fixed_row = cols
            fixed_lines.append(','.join(fixed_row))
        
        # Step 3: Load into pandas
        clean_csv = '\n'.join(fixed_lines)
        return pd.read_csv(StringIO(clean_csv))
    
    except Exception as e:
        print(f"Failed to parse CSV response: {e}")
        print(f"Raw content: {content}")
        return None

In [82]:
df = parse_llm_csv_response(response)
print(f"Number of rows: {len(df.index)}")
df.head(5)

Number of rows: 30


,product_id,product_brand,product_name,product_description,product_price,product_category,product_subcategory
0,1,Luxor,Luxor Premium T-Shirt,High-quality cotton T-shirt with elegant design,49.99,Clothing,T-shirts
1,2,Luxor,Luxor Luxe Pants,Stylish tailored pants crafted from fine mater...,129.99,Clothing,Pants
2,3,Luxor,Luxor Designer Shoes,Handcrafted leather shoes for formal occasions,199.99,Clothing,Shoes
3,4,Hermanos,Hermanos Classic T-Shirt,Comfortable and stylish T-shirt for everyday wear,24.99,Clothing,T-shirts
4,5,Hermanos,Hermanos Casual Pants,Durable pants suited for casual outings,49.99,Clothing,Pants


In [ ]:
df.to_csv("../data/products.csv", index=False)


## Customers dataset generation

In [50]:
# Experiment with a few data points before scaling up
rows = 100
prompt = f'''
Create a synthetic e-commerce customers dataset in csv format with {rows} rows.

The dataset should include the following columns:
customer_id, customer_city, customer_state, customer_zip, customer_country, customer_gender, customer_age, customer_class_quartile, customer_description

The customer_class_quartile should be one of the following: "Bottom 25%", "Lower Middle 25-50% ", "Upper Middle 50-75%", "Top 25%".
The customer_gender should be one of the following: "Male", "Female", "Other".
The customer_age should be an integer between 18 and 85.

These customers are from the United States in California, New York, and Florida.
Their cities and zip codes should be realistic.
The customer_description should be a short realistic description of the customer.

Each customer should be unique, and should be an actual specific customer.
Only include the csv data in the response with no backticks or other formatting.
'''.strip()
response = litellm.completion(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": prompt
    }]
)

print(response['choices'][0]['message']['content'])


customer_id,customer_city,customer_state,customer_zip,customer_country,customer_gender,customer_age,customer_class_quartile,customer_description
1,Los Angeles,CA,90001,USA,Female,34,Upper Middle 50-75%,"Marketing Manager living in LA, enjoys outdoor activities."
2,New York,NY,10001,USA,Male,45,Top 25%,"Finance executive from NYC, loves gourmet food and travel."
3,Miami,FL,33101,USA,Female,29,Lower Middle 25-50%,"Blogger based in Miami, passionate about fashion and beauty."
4,San Francisco,CA,94101,USA,Other,26,Upper Middle 50-75%,"Tech startup founder, into innovation and design."
5,Brooklyn,NY,11201,USA,Male,38,Top 25%,"Art curator in Brooklyn, enjoys visiting galleries and music festivals."
6,Orlando,FL,32801,USA,Female,22,Bottom 25%,"Student at UCF, loves theme parks and social media."
7,San Diego,CA,92101,USA,Female,39,Upper Middle 50-75%,"HR consultant based in SD, enjoys yoga and healthy living."
8,Queens,NY,11368,USA,Male,31,Lower Middle 25-50%,"Delivery driver in Queens, tries 

In [54]:
import csv

def parse_llm_csv_response2(response):
    """Parse CSV response from LLM with misquoted fields and return a clean DataFrame."""
    try:
        content = response['choices'][0]['message']['content']

        # Step 1: Extract CSV content
        if '```csv' in content:
            start = content.find('```csv') + 6
            end = content.find('```', start)
            csv_data = content[start:end].strip()
        elif '```' in content:
            start = content.find('```') + 3
            end = content.find('```', start)
            csv_data = content[start:end].strip()
        else:
            csv_data = content.strip()

        # Step 2: Normalize using CSV reader with intelligent parsing
        reader = csv.reader(StringIO(csv_data), skipinitialspace=True)
        rows = list(reader)
        expected_col_count = len(rows[0])
        fixed_rows = [rows[0]]  # header

        for row in rows[1:]:
            if len(row) == expected_col_count:
                fixed_rows.append(row)
            else:
                # Use csv.reader to attempt parsing again with quoting=csv.QUOTE_MINIMAL
                # fallback: rebuild line and re-parse with stricter rules
                joined_line = ','.join(row)
                try:
                    parsed_row = next(csv.reader([joined_line], quotechar='"', skipinitialspace=True))
                    if len(parsed_row) == expected_col_count:
                        fixed_rows.append(parsed_row)
                    else:
                        raise ValueError("Unfixable row")
                except:
                    print(f"Skipping malformed row: {row}")

        # Step 3: Convert to CSV string then DataFrame
        output = StringIO()
        writer = csv.writer(output)
        writer.writerows(fixed_rows)
        output.seek(0)
        return pd.read_csv(output)

    except Exception as e:
        print(f"Failed to parse CSV response: {e}")
        print(f"Raw content: {content[:1000]}...")  # avoid dumping everything
        return None

In [55]:
df = parse_llm_csv_response(response)
print(f"Number of rows: {len(df.index)}")
df.head(5)

Number of rows: 100


,customer_id,customer_city,customer_state,customer_zip,customer_country,customer_gender,customer_age,customer_class_quartile,customer_description
0,1,Los Angeles,CA,90001,USA,Female,34,Upper Middle 50-75%,"Marketing Manager living in LA, enjoys outdoor..."
1,2,New York,NY,10001,USA,Male,45,Top 25%,"Finance executive from NYC, loves gourmet food..."
2,3,Miami,FL,33101,USA,Female,29,Lower Middle 25-50%,"Blogger based in Miami, passionate about fashi..."
3,4,San Francisco,CA,94101,USA,Other,26,Upper Middle 50-75%,"Tech startup founder, into innovation and design."
4,5,Brooklyn,NY,11201,USA,Male,38,Top 25%,"Art curator in Brooklyn, enjoys visiting galle..."


In [56]:
df.to_csv("../data/customers.csv", index=False)

## Orders dataset generation

In [74]:
# Experiment with a few data points before scaling up
rows = 2000
customers_df = pd.read_csv("../data/customers.csv")
products_df = pd.read_csv("../data/products.csv")

prompt = f'''
customers dataset:
{customers_df}
---
products dataset:
{products_df}
---
Create a synthetic e-commerce orders dataset in csv format with {rows} rows.

The dataset should include the following columns:
order_id, order_seq_nbr, customer_id, product_id, order_date

The order_id should be a unique identifier for the order.
The order_seq_nbr should be a sequential number for the order based on the product data if there are multiple products in the order.
The customer_id should be the customer_id for the customer from the customers dataset.
The product_id should be the product_id for the product from the products dataset.
The order_date should be a date in the past 3 years.

Each order should be unique, and should be an actual specific order.
Use the customer information to create a realistic order.
Only include the csv data in the response with no backticks or other formatting.
'''.strip()
response = litellm.completion(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": prompt
    }]
)

print(response['choices'][0]['message']['content'])

order_id,order_seq_nbr,customer_id,product_id,order_date
1,1,1,3,2021-10-12
1,2,1,9,2021-10-12
2,1,2,1,2021-11-22
3,1,3,24,2021-12-05
4,1,4,6,2021-09-15
4,2,4,21,2021-09-15
5,1,5,10,2021-08-30
6,1,6,15,2021-07-19
6,2,6,8,2021-07-19
7,1,7,4,2022-01-10
8,1,8,29,2022-02-14
9,1,9,19,2022-03-11
10,1,10,11,2022-04-22
10,2,10,20,2022-04-22
11,1,11,18,2022-05-26
12,1,12,27,2022-06-17
13,1,13,5,2022-07-08
14,1,14,16,2022-08-02
15,1,15,12,2022-09-25
16,1,16,25,2022-10-09
17,1,17,26,2022-11-01
18,1,18,2,2023-01-15
19,1,19,7,2023-02-08
19,2,19,14,2023-02-08
20,1,20,30,2023-03-30
21,1,21,24,2023-04-12
22,1,22,15,2023-05-15
22,2,22,3,2023-05-15
23,1,23,13,2023-06-18
24,1,24,1,2023-07-04
24,2,24,9,2023-07-04
25,1,25,6,2023-08-22
26,1,26,23,2023-09-10
27,1,27,2,2021-05-30
28,1,28,8,2022-08-29
29,1,29,22,2022-09-15
30,1,30,3,2021-10-01
30,2,30,14,2021-10-01
31,1,31,28,2022-12-30
32,1,32,16,2023-01-12
33,1,33,10,2023-02-24
34,1,34,4,2021-07-20
35,1,35,5,2021-06-16
36,1,36,19,2022-03-18
37,1,37,12,2022-0

In [75]:
df = parse_llm_csv_response(response)
print(f"Number of rows: {len(df.index)}")
df.head(5)

Number of rows: 1086


,order_id,order_seq_nbr,customer_id,product_id,order_date
0,1,1,1,3,2021-10-12
1,1,2,1,9,2021-10-12
2,2,1,2,1,2021-11-22
3,3,1,3,24,2021-12-05
4,4,1,4,6,2021-09-15


In [76]:
df.to_csv("../data/orders2.csv", index=False)

In [77]:
# Load the two orders datasets
orders1 = pd.read_csv("../data/orders.csv")
orders2 = pd.read_csv("../data/orders2.csv")

# Concatenate the two DataFrames
merged_orders = pd.concat([orders1, orders2], ignore_index=True)

# Replace order_id with a new unique key (sequential starting from 1)
merged_orders = merged_orders.drop(columns=['order_id'])
merged_orders.insert(0, 'order_id', range(1, len(merged_orders) + 1))

# Save the merged DataFrame to a new CSV file
merged_orders.to_csv("../data/orders_merged.csv", index=False)

# Optionally, display the first few rows
merged_orders.head()


,order_id,order_seq_nbr,customer_id,product_id,order_date
0,1,1,1,1,2021-10-15
1,2,1,1,3,2021-10-15
2,3,1,2,2,2021-10-16
3,4,1,2,4,2021-10-16
4,5,1,3,5,2021-10-17


In [78]:
merged_orders.to_csv("../data/orders.csv", index=False)